# * VINSIGHT : Data Issue

## Parameter

In [1]:
import configparser
import datetime as dt
import pandas as pd
import numpy as np
import xlrd
import oracledb
import re
import FN_Monitoring as fn

config = configparser.ConfigParser()
config.read('../../my_config.ini')
config.sections()

TDMDBPR_user = config['TDMDBPR']['username']
TDMDBPR_pwd = config['TDMDBPR']['password']
TDMDBPR_db = config['TDMDBPR']['db']
TDMDBPR_host = config['TDMDBPR']['host']
TDMDBPR_port = config['TDMDBPR']['port']

AKPIPRD_user = config['AKPIPRD']['username']
AKPIPRD_pwd = config['AKPIPRD']['password']
AKPIPRD_db = config['AKPIPRD']['db']
AKPIPRD_host = config['AKPIPRD']['host']
AKPIPRD_port = config['AKPIPRD']['port']

curr_dt = dt.datetime.now().date()
str_curr_dt = curr_dt.strftime('%Y%m%d')

## Issue History

### Prepaid Active Caller 30D Rolling

In [2]:
''' Execute transaction '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20240101
print(f'\nParameter input...\n')
print(f'   -> Data as of {curr_datetime}')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_MTH, TM_KEY_DAY --, CENTER, PRODUCT_GRP, METRIC_GRP
        , METRIC_CD, METRIC_NAME --, AREA_TYPE, AREA_CD
        , AREA_NAME, ACTUAL_AS_OF, ACTUAL_SNAP, ACTUAL_AGG_MTH --, TARGET_SNAP
        , PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    --WHERE METRIC_CD = 'TB1S000600' --Prepaid Active Caller : TMH
    WHERE METRIC_CD IN (
         'TB1S000600' --Prepaid Active Caller : TMH
         , 'DB1S000600' --Prepaid Active Caller : DTAC
         )
    AND AREA_CD = 'P'
    AND TM_KEY_DAY >= {v_start_date} 
    ORDER BY TM_KEY_DAY, CENTER, PRODUCT_GRP, METRIC_GRP, METRIC_CD, AREA_TYPE, AREA_CD
""")


try:
    # Get : Timestamp
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    # Display
    # txn_df = chk_src_df.copy()
    # # txn_df = txn_df.fillna(0, inplace=True)
    # # txn_df = txn_df.replace(np.nan, None)
    # mod_col_list = txn_df.iloc[:, 6:8].columns.tolist()
    # for col in mod_col_list:
    #     txn_df[col] = txn_df[col].apply(lambda x: format(x, ',.0f'))
    # print(f'\n{txn_df.to_string(max_cols=10)}') #max_rows=1000
    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...

   -> Data as of 2025-05-16, 10:58:06
   -> v_start_date: 20240101


DatabaseError: ORA-02391: exceeded simultaneous SESSIONS_PER_USER limit
Help: https://docs.oracle.com/error-help/db/ora-02391/

In [13]:
''' TB1S000600 : Prepaid Active Caller 30D Rolling : TMH '''

v_metric_cd = 'TB1S000600'
# v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250401
# print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB1S000600_day_df = chk_src_df.copy()
TB1S000600_day_df = TB1S000600_day_df.loc[TB1S000600_day_df['METRIC_CD']==v_metric_cd]
# TB1S000600_day_df = TB1S000600_day_df.loc[TB1S000600_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB1S000600_day_df = TB1S000600_day_df.fillna(0).sort_values(by=['TM_KEY_DAY', 'METRIC_CD']).reset_index(drop=True)

mod_col_list = TB1S000600_day_df.iloc[:, 6:8].columns.tolist()
for col in mod_col_list:
    TB1S000600_day_df[col] = TB1S000600_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB1S000600_day_df

,TM_KEY_MTH,TM_KEY_DAY,METRIC_CD,METRIC_NAME,AREA_NAME,ACTUAL_AS_OF,ACTUAL_SNAP,ACTUAL_AGG_MTH,PPN_TM
0,202401,20240101,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,Nationwide,20240101.0,"14,140,668","14,140,668",2025-05-06 23:00:11
1,202401,20240102,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,Nationwide,20240102.0,"14,182,166","14,182,166",2025-05-06 23:00:11
2,202401,20240103,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,Nationwide,20240103.0,"14,243,980","14,243,980",2025-05-06 23:00:11
3,202401,20240104,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,Nationwide,20240104.0,"14,242,774","14,242,774",2025-05-06 23:00:11
4,202401,20240105,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,Nationwide,20240105.0,"14,266,164","14,266,164",2025-05-06 23:00:11
...,...,...,...,...,...,...,...,...,...
486,202505,20250501,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,Nationwide,20250501.0,"16,730,396","16,730,396",2025-05-06 23:00:11
487,202505,20250502,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,Nationwide,20250502.0,"16,798,106","16,798,106",2025-05-06 23:00:11
488,202505,20250503,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,Nationwide,20250503.0,"16,796,420","16,796,420",2025-05-06 23:00:11
489,202505,20250504,TB1S000600,Prepaid Active Caller 30D Rolling : TMH,Nationwide,20250504.0,"16,788,470","16,788,470",2025-05-06 23:00:11


In [14]:
''' DB1S000600 : Prepaid Active Caller 30D Rolling : DTAC '''

v_metric_cd = 'DB1S000600'
# v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250401
# print(f'   -> v_tm_key_day: {v_tm_key_day}')

DB1S000600_day_df = chk_src_df.copy()
DB1S000600_day_df = DB1S000600_day_df.loc[DB1S000600_day_df['METRIC_CD']==v_metric_cd]
# DB1S000600_day_df = DB1S000600_day_df.loc[DB1S000600_day_df['TM_KEY_DAY']>=v_tm_key_day]
DB1S000600_day_df = DB1S000600_day_df.fillna(0).sort_values(by=['TM_KEY_DAY', 'METRIC_CD']).reset_index(drop=True)

mod_col_list = DB1S000600_day_df.iloc[:, 6:8].columns.tolist()
for col in mod_col_list:
    DB1S000600_day_df[col] = DB1S000600_day_df[col].apply(lambda x: format(x, ',.0f'))
    
DB1S000600_day_df

,TM_KEY_MTH,TM_KEY_DAY,METRIC_CD,METRIC_NAME,AREA_NAME,ACTUAL_AS_OF,ACTUAL_SNAP,ACTUAL_AGG_MTH,PPN_TM
0,202401,20240101,DB1S000600,Prepaid Active Caller 30D Rolling : DTAC,Nationwide,0.0,0,0,2025-05-06 23:00:11
1,202401,20240102,DB1S000600,Prepaid Active Caller 30D Rolling : DTAC,Nationwide,0.0,0,0,2025-05-06 23:00:11
2,202401,20240103,DB1S000600,Prepaid Active Caller 30D Rolling : DTAC,Nationwide,0.0,0,0,2025-05-06 23:00:11
3,202401,20240104,DB1S000600,Prepaid Active Caller 30D Rolling : DTAC,Nationwide,0.0,0,0,2025-05-06 23:00:11
4,202401,20240105,DB1S000600,Prepaid Active Caller 30D Rolling : DTAC,Nationwide,0.0,0,0,2025-05-06 23:00:11
...,...,...,...,...,...,...,...,...,...
486,202505,20250501,DB1S000600,Prepaid Active Caller 30D Rolling : DTAC,Nationwide,20250501.0,"9,838,098","9,838,098",2025-05-06 23:00:11
487,202505,20250502,DB1S000600,Prepaid Active Caller 30D Rolling : DTAC,Nationwide,20250502.0,"9,851,269","9,851,269",2025-05-06 23:00:11
488,202505,20250503,DB1S000600,Prepaid Active Caller 30D Rolling : DTAC,Nationwide,20250503.0,"9,843,822","9,843,822",2025-05-06 23:00:11
489,202505,20250504,DB1S000600,Prepaid Active Caller 30D Rolling : DTAC,Nationwide,20250504.0,"9,832,807","9,832,807",2025-05-06 23:00:11


### Area 080 : DTAC

In [2]:
''' Execute transaction '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20250101
print(f'\nParameter input...\n')
print(f'   -> Data as of {curr_datetime}')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_MTH, TM_KEY_DAY, METRIC_CD, METRIC_NAME
        , SUM(CASE WHEN AREA_CD = '080' THEN ACTUAL_SNAP END) ACTUAL_080
        , SUM(CASE WHEN AREA_CD = '080' THEN TARGET_SNAP END) TARGET_080
        , SUM(CASE WHEN AREA_CD = '016' THEN ACTUAL_SNAP END) ACTUAL_016
        , SUM(CASE WHEN AREA_CD = '016' THEN TARGET_SNAP END) TARGET_016
        , SUM(CASE WHEN AREA_CD = '040' THEN ACTUAL_SNAP END) ACTUAL_040
        , SUM(CASE WHEN AREA_CD = '040' THEN TARGET_SNAP END) TARGET_040
        , MAX(ACTUAL_AS_OF) ACTUAL_AS_OF, MAX(PPN_TM) PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD = 'DB1R000100' --Prepaid Revenue : DTAC
    --WHERE METRIC_CD = 'DB2R000100' --Postpaid Revenue : DTAC
    --WHERE METRIC_CD = 'DB2R010100' --Postpaid Revenue B2C : DTAC
    --WHERE METRIC_CD = 'DB2R020100' --Postpaid Revenue B2B : DTAC
    AND AREA_CD IN ('016', '040', '080')
    AND TM_KEY_DAY >= {v_start_date} 
    GROUP BY TM_KEY_MTH, TM_KEY_DAY, METRIC_CD, METRIC_NAME
    ORDER BY TM_KEY_DAY, METRIC_CD
""")


try:
    # Get : Timestamp
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    # Display
    # txn_df = chk_src_df.copy()
    # txn_df = txn_df.replace(np.nan, 0)
    # mod_col_list = txn_df.iloc[:, 4:10].columns.tolist()
    # for col in mod_col_list:
    #     txn_df[col] = txn_df[col].apply(lambda x: format(x, ',.0f'))
    # print(f'\n{txn_df.to_string(max_cols=100)}')
    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...

   -> Data as of 2025-05-22, 09:51:44
   -> v_start_date: 20250101


In [4]:
''' Sample Data '''

txn_df = chk_src_df.copy()
txn_df = txn_df.replace(np.nan, 0)
mod_col_list = txn_df.iloc[:, 4:10].columns.tolist()
for col in mod_col_list:
    txn_df[col] = txn_df[col].apply(lambda x: format(x, ',.0f'))

# print(f'\n{txn_df.to_string(max_cols=100)}')
txn_df

C:\Users\Narut4\AppData\Local\Temp\ipykernel_24672\3747703875.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  txn_df = txn_df.replace(np.nan, 0)


,TM_KEY_MTH,TM_KEY_DAY,METRIC_CD,METRIC_NAME,ACTUAL_080,TARGET_080,ACTUAL_016,TARGET_016,ACTUAL_040,TARGET_040,ACTUAL_AS_OF,PPN_TM
0,202501,20250101,DB1R000100,Prepaid Revenue : DTAC,"1,458,637",0,0,0,0,0,20250101,2025-05-21 05:46:58
1,202501,20250102,DB1R000100,Prepaid Revenue : DTAC,"1,378,370",0,0,0,0,0,20250102,2025-05-21 05:46:58
2,202501,20250103,DB1R000100,Prepaid Revenue : DTAC,"1,330,136",0,0,0,0,0,20250103,2025-05-21 05:46:58
3,202501,20250104,DB1R000100,Prepaid Revenue : DTAC,"1,229,670",0,0,0,0,0,20250104,2025-05-21 05:46:58
4,202501,20250105,DB1R000100,Prepaid Revenue : DTAC,"1,312,542",0,0,0,0,0,20250105,2025-05-21 05:46:58
...,...,...,...,...,...,...,...,...,...,...,...,...
135,202505,20250516,DB1R000100,Prepaid Revenue : DTAC,"1,234,064","1,221,957",0,0,0,0,20250516,2025-05-21 05:46:58
136,202505,20250517,DB1R000100,Prepaid Revenue : DTAC,"1,205,304","1,205,382",0,0,0,0,20250517,2025-05-21 05:46:58
137,202505,20250518,DB1R000100,Prepaid Revenue : DTAC,"1,295,234","1,162,579",0,0,0,0,20250518,2025-05-21 05:46:58
138,202505,20250519,DB1R000100,Prepaid Revenue : DTAC,"1,227,031","1,160,581",0,0,0,0,20250519,2025-05-21 05:46:58


In [3]:
jan25_df = chk_src_df.loc[chk_src_df['TM_KEY_MTH']==202501].copy()
jan25_df = jan25_df.replace(np.nan, 0)
mod_col_list = jan25_df.iloc[:, 4:10].columns.tolist()
for col in mod_col_list:
    jan25_df[col] = jan25_df[col].apply(lambda x: format(x, ',.0f'))
    
jan25_df

C:\Users\Narut4\AppData\Local\Temp\ipykernel_18160\1336843066.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  jan25_df = jan25_df.replace(np.nan, 0)


,TM_KEY_MTH,TM_KEY_DAY,METRIC_CD,METRIC_NAME,ACTUAL_080,TARGET_080,ACTUAL_016,TARGET_016,ACTUAL_040,TARGET_040,ACTUAL_AS_OF,PPN_TM
0,202501,20250101,DB1R000100,Prepaid Revenue : DTAC,"1,458,637","1,425,288",0,0,0,0,20250101,2025-05-22 00:56:22
1,202501,20250102,DB1R000100,Prepaid Revenue : DTAC,"1,378,370","1,457,336",0,0,0,0,20250102,2025-05-22 00:56:22
2,202501,20250103,DB1R000100,Prepaid Revenue : DTAC,"1,330,136","1,413,942",0,0,0,0,20250103,2025-05-22 00:56:22
3,202501,20250104,DB1R000100,Prepaid Revenue : DTAC,"1,229,670","1,358,850",0,0,0,0,20250104,2025-05-22 00:56:22
4,202501,20250105,DB1R000100,Prepaid Revenue : DTAC,"1,312,542","1,362,544",0,0,0,0,20250105,2025-05-22 00:56:22
5,202501,20250106,DB1R000100,Prepaid Revenue : DTAC,"1,257,104","1,290,379",0,0,0,0,20250106,2025-05-22 00:56:22
6,202501,20250107,DB1R000100,Prepaid Revenue : DTAC,"1,264,117","1,248,284",0,0,0,0,20250107,2025-05-22 00:56:22
7,202501,20250108,DB1R000100,Prepaid Revenue : DTAC,"1,205,018","1,265,016",0,0,0,0,20250108,2025-05-22 00:56:22
8,202501,20250109,DB1R000100,Prepaid Revenue : DTAC,"1,075,793","1,219,491",0,0,0,0,20250109,2025-05-22 00:56:22
9,202501,20250110,DB1R000100,Prepaid Revenue : DTAC,"1,146,732","1,246,177",0,0,0,0,20250110,2025-05-22 00:56:22


In [4]:
feb25_df = chk_src_df.loc[chk_src_df['TM_KEY_MTH']==202502].copy()
feb25_df = feb25_df.replace(np.nan, 0)
mod_col_list = feb25_df.iloc[:, 4:10].columns.tolist()
for col in mod_col_list:
    feb25_df[col] = feb25_df[col].apply(lambda x: format(x, ',.0f'))
    
feb25_df

C:\Users\Narut4\AppData\Local\Temp\ipykernel_18160\4083946966.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  feb25_df = feb25_df.replace(np.nan, 0)


,TM_KEY_MTH,TM_KEY_DAY,METRIC_CD,METRIC_NAME,ACTUAL_080,TARGET_080,ACTUAL_016,TARGET_016,ACTUAL_040,TARGET_040,ACTUAL_AS_OF,PPN_TM
31,202502,20250201,DB1R000100,Prepaid Revenue : DTAC,"1,537,712","1,457,974",0,0,0,0,20250201,2025-05-22 00:56:22
32,202502,20250202,DB1R000100,Prepaid Revenue : DTAC,"1,472,950","1,395,283",0,0,0,0,20250202,2025-05-22 00:56:22
33,202502,20250203,DB1R000100,Prepaid Revenue : DTAC,"1,361,083","1,376,123",0,0,0,0,20250203,2025-05-22 00:56:22
34,202502,20250204,DB1R000100,Prepaid Revenue : DTAC,"1,271,479","1,314,550",0,0,0,0,20250204,2025-05-22 00:56:22
35,202502,20250205,DB1R000100,Prepaid Revenue : DTAC,"1,291,841","1,309,221",0,0,0,0,20250205,2025-05-22 00:56:22
36,202502,20250206,DB1R000100,Prepaid Revenue : DTAC,"1,338,937","1,304,648",0,0,0,0,20250206,2025-05-22 00:56:22
37,202502,20250207,DB1R000100,Prepaid Revenue : DTAC,"1,230,067","1,273,728",0,0,0,0,20250207,2025-05-22 00:56:22
38,202502,20250208,DB1R000100,Prepaid Revenue : DTAC,"1,200,707","1,278,664",0,0,0,0,20250208,2025-05-22 00:56:22
39,202502,20250209,DB1R000100,Prepaid Revenue : DTAC,"1,169,782","1,214,903",0,0,0,0,20250209,2025-05-22 00:56:22
40,202502,20250210,DB1R000100,Prepaid Revenue : DTAC,"1,130,583","1,267,316",0,0,0,0,20250210,2025-05-22 00:56:22


In [5]:
mar25_df = chk_src_df.loc[chk_src_df['TM_KEY_MTH']==202503].copy()
mar25_df = mar25_df.replace(np.nan, 0)
mod_col_list = mar25_df.iloc[:, 4:10].columns.tolist()
for col in mod_col_list:
    mar25_df[col] = mar25_df[col].apply(lambda x: format(x, ',.0f'))
    
mar25_df

C:\Users\Narut4\AppData\Local\Temp\ipykernel_18160\3737428264.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  mar25_df = mar25_df.replace(np.nan, 0)


,TM_KEY_MTH,TM_KEY_DAY,METRIC_CD,METRIC_NAME,ACTUAL_080,TARGET_080,ACTUAL_016,TARGET_016,ACTUAL_040,TARGET_040,ACTUAL_AS_OF,PPN_TM
59,202503,20250301,DB1R000100,Prepaid Revenue : DTAC,"1,506,675","1,567,283",0,0,0,0,20250301,2025-05-22 00:56:22
60,202503,20250302,DB1R000100,Prepaid Revenue : DTAC,"1,546,780","1,476,098",0,0,0,0,20250302,2025-05-22 00:56:22
61,202503,20250303,DB1R000100,Prepaid Revenue : DTAC,"1,449,951","1,384,844",0,0,0,0,20250303,2025-05-22 00:56:22
62,202503,20250304,DB1R000100,Prepaid Revenue : DTAC,"1,348,670","1,327,416",0,0,0,0,20250304,2025-05-22 00:56:22
63,202503,20250305,DB1R000100,Prepaid Revenue : DTAC,"1,342,948","1,359,366",0,0,0,0,20250305,2025-05-22 00:56:22
64,202503,20250306,DB1R000100,Prepaid Revenue : DTAC,"1,279,234","1,349,882",0,0,0,0,20250306,2025-05-22 00:56:22
65,202503,20250307,DB1R000100,Prepaid Revenue : DTAC,"1,247,824","1,283,626",0,0,0,0,20250307,2025-05-22 00:56:22
66,202503,20250308,DB1R000100,Prepaid Revenue : DTAC,"1,248,182","1,238,708",0,0,0,0,20250308,2025-05-22 00:56:22
67,202503,20250309,DB1R000100,Prepaid Revenue : DTAC,"1,225,405","1,167,094",0,0,0,0,20250309,2025-05-22 00:56:22
68,202503,20250310,DB1R000100,Prepaid Revenue : DTAC,"1,149,640","1,152,327",0,0,0,0,20250310,2025-05-22 00:56:22


In [6]:
apr25_df = chk_src_df.loc[chk_src_df['TM_KEY_MTH']==202504].copy()
apr25_df = apr25_df.replace(np.nan, 0)
mod_col_list = apr25_df.iloc[:, 4:10].columns.tolist()
for col in mod_col_list:
    apr25_df[col] = apr25_df[col].apply(lambda x: format(x, ',.0f'))
    
apr25_df

C:\Users\Narut4\AppData\Local\Temp\ipykernel_18160\1557403921.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  apr25_df = apr25_df.replace(np.nan, 0)


,TM_KEY_MTH,TM_KEY_DAY,METRIC_CD,METRIC_NAME,ACTUAL_080,TARGET_080,ACTUAL_016,TARGET_016,ACTUAL_040,TARGET_040,ACTUAL_AS_OF,PPN_TM
90,202504,20250401,DB1R000100,Prepaid Revenue : DTAC,"1,575,177","1,339,649",0,0,0,0,20250401,2025-05-22 00:56:22
91,202504,20250402,DB1R000100,Prepaid Revenue : DTAC,"1,427,780","1,434,931",0,0,0,0,20250402,2025-05-22 00:56:22
92,202504,20250403,DB1R000100,Prepaid Revenue : DTAC,"1,463,032","1,298,233",0,0,0,0,20250403,2025-05-22 00:56:22
93,202504,20250404,DB1R000100,Prepaid Revenue : DTAC,"1,330,197","1,248,136",0,0,0,0,20250404,2025-05-22 00:56:22
94,202504,20250405,DB1R000100,Prepaid Revenue : DTAC,"1,326,432","1,332,687",0,0,0,0,20250405,2025-05-22 00:56:22
95,202504,20250406,DB1R000100,Prepaid Revenue : DTAC,"1,295,827","1,262,578",0,0,0,0,20250406,2025-05-22 00:56:22
96,202504,20250407,DB1R000100,Prepaid Revenue : DTAC,"1,197,326","1,217,703",0,0,0,0,20250407,2025-05-22 00:56:22
97,202504,20250408,DB1R000100,Prepaid Revenue : DTAC,"1,190,268","1,176,555",0,0,0,0,20250408,2025-05-22 00:56:22
98,202504,20250409,DB1R000100,Prepaid Revenue : DTAC,"1,138,984","1,159,320",0,0,0,0,20250409,2025-05-22 00:56:22
99,202504,20250410,DB1R000100,Prepaid Revenue : DTAC,"1,157,204","1,224,347",0,0,0,0,20250410,2025-05-22 00:56:22


In [7]:
may25_df = chk_src_df.loc[chk_src_df['TM_KEY_MTH']==202505].copy()
may25_df = may25_df.replace(np.nan, 0)
mod_col_list = may25_df.iloc[:, 4:10].columns.tolist()
for col in mod_col_list:
    may25_df[col] = may25_df[col].apply(lambda x: format(x, ',.0f'))
    
may25_df

C:\Users\Narut4\AppData\Local\Temp\ipykernel_18160\4129942621.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  may25_df = may25_df.replace(np.nan, 0)


,TM_KEY_MTH,TM_KEY_DAY,METRIC_CD,METRIC_NAME,ACTUAL_080,TARGET_080,ACTUAL_016,TARGET_016,ACTUAL_040,TARGET_040,ACTUAL_AS_OF,PPN_TM
120,202505,20250501,DB1R000100,Prepaid Revenue : DTAC,"1,635,728","1,536,962",0,0,0,0,20250501,2025-05-22 00:56:22
121,202505,20250502,DB1R000100,Prepaid Revenue : DTAC,"1,530,001","1,416,205",0,0,0,0,20250502,2025-05-22 00:56:22
122,202505,20250503,DB1R000100,Prepaid Revenue : DTAC,"1,401,946","1,356,224",0,0,0,0,20250503,2025-05-22 00:56:22
123,202505,20250504,DB1R000100,Prepaid Revenue : DTAC,"1,315,014","1,272,045",0,0,0,0,20250504,2025-05-22 00:56:22
124,202505,20250505,DB1R000100,Prepaid Revenue : DTAC,"1,314,119","1,312,972",0,0,0,0,20250505,2025-05-22 00:56:22
125,202505,20250506,DB1R000100,Prepaid Revenue : DTAC,"1,271,558","1,248,316",0,0,0,0,20250506,2025-05-22 00:56:22
126,202505,20250507,DB1R000100,Prepaid Revenue : DTAC,"1,283,653","1,241,571",0,0,0,0,20250507,2025-05-22 00:56:22
127,202505,20250508,DB1R000100,Prepaid Revenue : DTAC,"1,288,672","1,224,601",0,0,0,0,20250508,2025-05-22 00:56:22
128,202505,20250509,DB1R000100,Prepaid Revenue : DTAC,"1,172,239","1,180,005",0,0,0,0,20250509,2025-05-22 00:56:22
129,202505,20250510,DB1R000100,Prepaid Revenue : DTAC,"1,162,885","1,204,041",0,0,0,0,20250510,2025-05-22 00:56:22


### Prepaid Revenue : TMH

In [13]:
''' Execute transaction '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20250401
print(f'\nParameter input...\n')
print(f'   -> Data as of {curr_datetime}')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_MTH, TM_KEY_DAY --, CENTER, PRODUCT_GRP, METRIC_GRP
        , METRIC_CD, METRIC_NAME --, AREA_TYPE, AREA_CD
        , AREA_NAME, ACTUAL_AS_OF, ACTUAL_SNAP, ACTUAL_AGG_MTH --, TARGET_SNAP
        , PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD = 'TB1R000100' --Prepaid Revenue : TMH
    AND AREA_CD = 'P'
    AND TM_KEY_DAY >= {v_start_date} 
    ORDER BY TM_KEY_DAY, CENTER, PRODUCT_GRP, METRIC_GRP, METRIC_CD, AREA_TYPE, AREA_CD
""")


try:
    # Get : Timestamp
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    # Display
    txn_df = chk_src_df.copy()
    # txn_df = txn_df.replace(np.nan, 0)
    mod_col_list = txn_df.iloc[:, 6:8].columns.tolist()
    for col in mod_col_list:
        txn_df[col] = txn_df[col].apply(lambda x: format(x, ',.0f'))
    print(f'\n{txn_df.to_string(max_cols=10)}') #max_rows=1000
    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...

   -> Data as of 2025-05-02, 10:50:20
   -> v_start_date: 20250401

    TM_KEY_MTH  TM_KEY_DAY   METRIC_CD            METRIC_NAME   AREA_NAME  ACTUAL_AS_OF  ACTUAL_SNAP ACTUAL_AGG_MTH              PPN_TM
0       202504    20250401  TB1R000100  Prepaid Revenue : TMH  Nationwide      20250401  101,725,829    101,725,829 2025-05-02 08:51:18
1       202504    20250402  TB1R000100  Prepaid Revenue : TMH  Nationwide      20250402   92,314,263    194,040,092 2025-05-02 08:51:18
2       202504    20250403  TB1R000100  Prepaid Revenue : TMH  Nationwide      20250403   85,596,819    279,636,911 2025-05-02 08:51:18
3       202504    20250404  TB1R000100  Prepaid Revenue : TMH  Nationwide      20250404   85,954,445    365,591,356 2025-05-02 08:51:18
4       202504    20250405  TB1R000100  Prepaid Revenue : TMH  Nationwide      20250405   84,406,717    449,998,074 2025-05-02 08:51:18
5       202504    20250406  TB1R000100  Prepaid Revenue : TMH  Nationwide      20250406   80,94

### Monthly Revenue ARPU

In [14]:
''' Execute transaction '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20250101
print(f'\nParameter input...\n')
print(f'   -> Data as of {curr_datetime}')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_MTH, TM_KEY_DAY --, CENTER, PRODUCT_GRP, METRIC_GRP
        , METRIC_CD, METRIC_NAME --, AREA_TYPE, AREA_CD
        , AREA_NAME, ACTUAL_AS_OF, ACTUAL_SNAP, ACTUAL_AGG_MTH --, TARGET_SNAP
        , PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD IN (
        'B7R000102' --Prepaid ARPU
        , 'DB7R000102' --Prepaid ARPU : DTAC
        , 'TB7R000102' --Prepaid ARPU : TMH
        , 'B7R000101' --Postpaid ARPU
        , 'B7R010101' --Postpaid ARPU B2C
        , 'DB7R000101' --Postpaid ARPU : DTAC
        , 'DB7R010101' --Postpaid ARPU B2C : DTAC
        , 'TB7R000101' --Postpaid ARPU : TMH
        , 'TB7R010101' --Postpaid ARPU B2C : TMH
        , 'TB7R000103' --TOL ARPU
        , 'TB7R000104' --TVS ARPU
        )
    AND AREA_CD = 'P'
    AND TM_KEY_DAY >= {v_start_date} 
    --ORDER BY TM_KEY_DAY, CENTER, PRODUCT_GRP, METRIC_GRP, METRIC_CD, AREA_TYPE, AREA_CD
""")


try:
    # Get : Timestamp
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    # # Display
    # txn_df = chk_src_df.copy()
    # mod_col_list = txn_df.iloc[:, 6:8].columns.tolist()
    # for col in mod_col_list:
    #     txn_df[col] = txn_df[col].apply(lambda x: format(x, ',.0f'))
    # print(f'\n{txn_df.to_string(max_cols=10)}') #max_rows=1000

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...

   -> Data as of 2025-05-02, 10:50:51
   -> v_start_date: 20250101


In [15]:
''' B7R000102 : Prepaid ARPU '''

v_metric_cd = 'B7R000102'
# v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250401
print(f'\nB7R000102 : Prepaid ARPU...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
# print(f'   -> v_tm_key_day: {v_tm_key_day}')

B7R000102_day_df = chk_src_df.copy()
B7R000102_day_df = B7R000102_day_df.loc[B7R000102_day_df['METRIC_CD']==v_metric_cd]
# B7R000102_day_df = B7R000102_day_df.loc[B7R000102_day_df['TM_KEY_DAY']>=v_tm_key_day]
B7R000102_day_df = B7R000102_day_df.fillna(0).sort_values(by=['TM_KEY_DAY', 'METRIC_CD']).reset_index(drop=True)

mod_col_list = B7R000102_day_df.iloc[:, 6:8].columns.tolist()
for col in mod_col_list:
    B7R000102_day_df[col] = B7R000102_day_df[col].apply(lambda x: format(x, ',.1f'))
    
B7R000102_day_df


B7R000102 : Prepaid ARPU...

   -> v_metric_cd: B7R000102


,TM_KEY_MTH,TM_KEY_DAY,METRIC_CD,METRIC_NAME,AREA_NAME,ACTUAL_AS_OF,ACTUAL_SNAP,ACTUAL_AGG_MTH,PPN_TM
0,202501,20250101,B7R000102,Prepaid ARPU,Nationwide,20250101,172.9,172.9,2025-05-02 08:51:18
1,202501,20250102,B7R000102,Prepaid ARPU,Nationwide,20250102,172.8,172.8,2025-05-02 08:51:18
2,202501,20250103,B7R000102,Prepaid ARPU,Nationwide,20250103,173.2,173.2,2025-05-02 08:51:18
3,202501,20250104,B7R000102,Prepaid ARPU,Nationwide,20250104,173.4,173.4,2025-05-02 08:51:18
4,202501,20250105,B7R000102,Prepaid ARPU,Nationwide,20250105,173.5,173.5,2025-05-02 08:51:18
...,...,...,...,...,...,...,...,...,...
116,202504,20250427,B7R000102,Prepaid ARPU,Nationwide,20250427,151.5,151.5,2025-05-02 08:51:18
117,202504,20250428,B7R000102,Prepaid ARPU,Nationwide,20250428,151.5,151.5,2025-05-02 08:51:18
118,202504,20250429,B7R000102,Prepaid ARPU,Nationwide,20250428,0.0,151.5,2025-05-02 08:51:18
119,202504,20250430,B7R000102,Prepaid ARPU,Nationwide,20250428,0.0,151.5,2025-05-02 08:51:18


In [16]:
''' B7R010101 : Postpaid ARPU B2C '''

v_metric_cd = 'B7R010101'
# v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250401
print(f'\nB7R010101 : Postpaid ARPU B2C...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
# print(f'   -> v_tm_key_day: {v_tm_key_day}')

B7R010101_day_df = chk_src_df.copy()
B7R010101_day_df = B7R010101_day_df.loc[B7R010101_day_df['METRIC_CD']==v_metric_cd]
# B7R010101_day_df = B7R010101_day_df.loc[B7R010101_day_df['TM_KEY_DAY']>=v_tm_key_day]
B7R010101_day_df = B7R010101_day_df.fillna(0).sort_values(by=['TM_KEY_DAY', 'METRIC_CD']).reset_index(drop=True)

mod_col_list = B7R010101_day_df.iloc[:, 6:8].columns.tolist()
for col in mod_col_list:
    B7R010101_day_df[col] = B7R010101_day_df[col].apply(lambda x: format(x, ',.1f'))
    
B7R010101_day_df


B7R010101 : Postpaid ARPU B2C...

   -> v_metric_cd: B7R010101


,TM_KEY_MTH,TM_KEY_DAY,METRIC_CD,METRIC_NAME,AREA_NAME,ACTUAL_AS_OF,ACTUAL_SNAP,ACTUAL_AGG_MTH,PPN_TM
0,202501,20250101,B7R010101,Postpaid ARPU B2C,Nationwide,20250101,416.8,416.8,2025-05-02 08:51:18
1,202501,20250102,B7R010101,Postpaid ARPU B2C,Nationwide,20250102,417.0,417.0,2025-05-02 08:51:18
2,202501,20250103,B7R010101,Postpaid ARPU B2C,Nationwide,20250103,417.0,417.0,2025-05-02 08:51:18
3,202501,20250104,B7R010101,Postpaid ARPU B2C,Nationwide,20250104,417.0,417.0,2025-05-02 08:51:18
4,202501,20250105,B7R010101,Postpaid ARPU B2C,Nationwide,20250105,416.7,416.7,2025-05-02 08:51:18
...,...,...,...,...,...,...,...,...,...
116,202504,20250427,B7R010101,Postpaid ARPU B2C,Nationwide,20250427,416.7,416.7,2025-05-02 08:51:18
117,202504,20250428,B7R010101,Postpaid ARPU B2C,Nationwide,20250428,392.3,392.3,2025-05-02 08:51:18
118,202504,20250429,B7R010101,Postpaid ARPU B2C,Nationwide,20250428,0.0,392.3,2025-05-02 08:51:18
119,202504,20250430,B7R010101,Postpaid ARPU B2C,Nationwide,20250428,0.0,392.3,2025-05-02 08:51:18


In [17]:
''' TB7R010101 : Postpaid ARPU B2C : TMH '''

v_metric_cd = 'TB7R010101'
# v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250401
print(f'\nTB7R010101 : Postpaid ARPU B2C : TMH...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
# print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB7R010101_day_df = chk_src_df.copy()
TB7R010101_day_df = TB7R010101_day_df.loc[TB7R010101_day_df['METRIC_CD']==v_metric_cd]
# TB7R010101_day_df = TB7R010101_day_df.loc[TB7R010101_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB7R010101_day_df = TB7R010101_day_df.fillna(0).sort_values(by=['TM_KEY_DAY', 'METRIC_CD']).reset_index(drop=True)

mod_col_list = TB7R010101_day_df.iloc[:, 6:8].columns.tolist()
for col in mod_col_list:
    TB7R010101_day_df[col] = TB7R010101_day_df[col].apply(lambda x: format(x, ',.1f'))
    
TB7R010101_day_df


TB7R010101 : Postpaid ARPU B2C : TMH...

   -> v_metric_cd: TB7R010101


,TM_KEY_MTH,TM_KEY_DAY,METRIC_CD,METRIC_NAME,AREA_NAME,ACTUAL_AS_OF,ACTUAL_SNAP,ACTUAL_AGG_MTH,PPN_TM
0,202501,20250101,TB7R010101,Postpaid ARPU B2C : TMH,Nationwide,20250101,371.6,371.6,2025-05-02 08:51:18
1,202501,20250102,TB7R010101,Postpaid ARPU B2C : TMH,Nationwide,20250102,372.0,372.0,2025-05-02 08:51:18
2,202501,20250103,TB7R010101,Postpaid ARPU B2C : TMH,Nationwide,20250103,372.0,372.0,2025-05-02 08:51:18
3,202501,20250104,TB7R010101,Postpaid ARPU B2C : TMH,Nationwide,20250104,372.0,372.0,2025-05-02 08:51:18
4,202501,20250105,TB7R010101,Postpaid ARPU B2C : TMH,Nationwide,20250105,371.9,371.9,2025-05-02 08:51:18
...,...,...,...,...,...,...,...,...,...
116,202504,20250427,TB7R010101,Postpaid ARPU B2C : TMH,Nationwide,20250427,371.7,371.7,2025-05-02 08:51:18
117,202504,20250428,TB7R010101,Postpaid ARPU B2C : TMH,Nationwide,20250428,333.7,333.7,2025-05-02 08:51:18
118,202504,20250429,TB7R010101,Postpaid ARPU B2C : TMH,Nationwide,20250429,333.9,333.9,2025-05-02 08:51:18
119,202504,20250430,TB7R010101,Postpaid ARPU B2C : TMH,Nationwide,20250429,0.0,333.9,2025-05-02 08:51:18


In [18]:
''' DB7R010101 : Postpaid ARPU B2C : DTAC '''

v_metric_cd = 'DB7R010101'
# v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250401
print(f'\nDB7R010101 : Postpaid ARPU B2C : DTAC...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
# print(f'   -> v_tm_key_day: {v_tm_key_day}')

DB7R010101_day_df = chk_src_df.copy()
DB7R010101_day_df = DB7R010101_day_df.loc[DB7R010101_day_df['METRIC_CD']==v_metric_cd]
# DB7R010101_day_df = DB7R010101_day_df.loc[DB7R010101_day_df['TM_KEY_DAY']>=v_tm_key_day]
DB7R010101_day_df = DB7R010101_day_df.fillna(0).sort_values(by=['TM_KEY_DAY', 'METRIC_CD']).reset_index(drop=True)

mod_col_list = DB7R010101_day_df.iloc[:, 6:8].columns.tolist()
for col in mod_col_list:
    DB7R010101_day_df[col] = DB7R010101_day_df[col].apply(lambda x: format(x, ',.1f'))
    
DB7R010101_day_df


DB7R010101 : Postpaid ARPU B2C : DTAC...

   -> v_metric_cd: DB7R010101


,TM_KEY_MTH,TM_KEY_DAY,METRIC_CD,METRIC_NAME,AREA_NAME,ACTUAL_AS_OF,ACTUAL_SNAP,ACTUAL_AGG_MTH,PPN_TM
0,202501,20250101,DB7R010101,Postpaid ARPU B2C : DTAC,Nationwide,20250101,494.7,494.7,2025-05-02 08:51:18
1,202501,20250102,DB7R010101,Postpaid ARPU B2C : DTAC,Nationwide,20250102,494.5,494.5,2025-05-02 08:51:18
2,202501,20250103,DB7R010101,Postpaid ARPU B2C : DTAC,Nationwide,20250103,494.6,494.6,2025-05-02 08:51:18
3,202501,20250104,DB7R010101,Postpaid ARPU B2C : DTAC,Nationwide,20250104,494.4,494.4,2025-05-02 08:51:18
4,202501,20250105,DB7R010101,Postpaid ARPU B2C : DTAC,Nationwide,20250105,494.1,494.1,2025-05-02 08:51:18
...,...,...,...,...,...,...,...,...,...
116,202504,20250427,DB7R010101,Postpaid ARPU B2C : DTAC,Nationwide,20250427,496.4,496.4,2025-05-02 08:51:18
117,202504,20250428,DB7R010101,Postpaid ARPU B2C : DTAC,Nationwide,20250428,496.2,496.2,2025-05-02 08:51:18
118,202504,20250429,DB7R010101,Postpaid ARPU B2C : DTAC,Nationwide,20250428,0.0,496.2,2025-05-02 08:51:18
119,202504,20250430,DB7R010101,Postpaid ARPU B2C : DTAC,Nationwide,20250428,0.0,496.2,2025-05-02 08:51:18


In [19]:
''' TB7R000103 : TOL ARPU '''

v_metric_cd = 'TB7R000103'
# v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250401
print(f'\nTB7R000103 : TOL ARPU...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
# print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB7R000103_day_df = chk_src_df.copy()
TB7R000103_day_df = TB7R000103_day_df.loc[TB7R000103_day_df['METRIC_CD']==v_metric_cd]
# TB7R000103_day_df = TB7R000103_day_df.loc[TB7R000103_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB7R000103_day_df = TB7R000103_day_df.fillna(0).sort_values(by=['TM_KEY_DAY', 'METRIC_CD']).reset_index(drop=True)

mod_col_list = TB7R000103_day_df.iloc[:, 6:8].columns.tolist()
for col in mod_col_list:
    TB7R000103_day_df[col] = TB7R000103_day_df[col].apply(lambda x: format(x, ',.1f'))
    
TB7R000103_day_df


TB7R000103 : TOL ARPU...

   -> v_metric_cd: TB7R000103


,TM_KEY_MTH,TM_KEY_DAY,METRIC_CD,METRIC_NAME,AREA_NAME,ACTUAL_AS_OF,ACTUAL_SNAP,ACTUAL_AGG_MTH,PPN_TM
0,202501,20250101,TB7R000103,TOL ARPU,Nationwide,20250101,510.9,510.9,2025-05-02 08:51:18
1,202501,20250102,TB7R000103,TOL ARPU,Nationwide,20250102,510.8,510.8,2025-05-02 08:51:18
2,202501,20250103,TB7R000103,TOL ARPU,Nationwide,20250103,510.9,510.9,2025-05-02 08:51:18
3,202501,20250104,TB7R000103,TOL ARPU,Nationwide,20250104,510.8,510.8,2025-05-02 08:51:18
4,202501,20250105,TB7R000103,TOL ARPU,Nationwide,20250105,510.6,510.6,2025-05-02 08:51:18
...,...,...,...,...,...,...,...,...,...
116,202504,20250427,TB7R000103,TOL ARPU,Nationwide,20250427,514.3,514.3,2025-05-02 08:51:18
117,202504,20250428,TB7R000103,TOL ARPU,Nationwide,20250428,513.3,513.3,2025-05-02 08:51:18
118,202504,20250429,TB7R000103,TOL ARPU,Nationwide,20250428,0.0,513.3,2025-05-02 08:51:18
119,202504,20250430,TB7R000103,TOL ARPU,Nationwide,20250428,0.0,513.3,2025-05-02 08:51:18


In [20]:
''' TB7R000104 : TVS ARPU '''

v_metric_cd = 'TB7R000104'
# v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250401
print(f'\nTB7R000104 : Postpaid ARPU B2C...\n')
print(f'   -> v_metric_cd: {v_metric_cd}')
# print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB7R000104_day_df = chk_src_df.copy()
TB7R000104_day_df = TB7R000104_day_df.loc[TB7R000104_day_df['METRIC_CD']==v_metric_cd]
# TB7R000104_day_df = TB7R000104_day_df.loc[TB7R000104_day_df['TM_KEY_DAY']>=v_tm_key_day]
TB7R000104_day_df = TB7R000104_day_df.fillna(0).sort_values(by=['TM_KEY_DAY', 'METRIC_CD']).reset_index(drop=True)

mod_col_list = TB7R000104_day_df.iloc[:, 6:8].columns.tolist()
for col in mod_col_list:
    TB7R000104_day_df[col] = TB7R000104_day_df[col].apply(lambda x: format(x, ',.1f'))
    
TB7R000104_day_df


TB7R000104 : Postpaid ARPU B2C...

   -> v_metric_cd: TB7R000104


,TM_KEY_MTH,TM_KEY_DAY,METRIC_CD,METRIC_NAME,AREA_NAME,ACTUAL_AS_OF,ACTUAL_SNAP,ACTUAL_AGG_MTH,PPN_TM
0,202501,20250101,TB7R000104,TVS ARPU,Nationwide,20250101,260.4,260.4,2025-05-02 08:51:18
1,202501,20250102,TB7R000104,TVS ARPU,Nationwide,20250102,259.8,259.8,2025-05-02 08:51:18
2,202501,20250103,TB7R000104,TVS ARPU,Nationwide,20250103,259.8,259.8,2025-05-02 08:51:18
3,202501,20250104,TB7R000104,TVS ARPU,Nationwide,20250104,259.9,259.9,2025-05-02 08:51:18
4,202501,20250105,TB7R000104,TVS ARPU,Nationwide,20250105,259.9,259.9,2025-05-02 08:51:18
...,...,...,...,...,...,...,...,...,...
116,202504,20250427,TB7R000104,TVS ARPU,Nationwide,20250427,259.0,259.0,2025-05-02 08:51:18
117,202504,20250428,TB7R000104,TVS ARPU,Nationwide,20250428,258.0,258.0,2025-05-02 08:51:18
118,202504,20250429,TB7R000104,TVS ARPU,Nationwide,20250428,0.0,258.0,2025-05-02 08:51:18
119,202504,20250430,TB7R000104,TVS ARPU,Nationwide,20250428,0.0,258.0,2025-05-02 08:51:18


## * Revenue Corporate (C, P)

In [2]:
''' Execute transaction '''


# Input parameter
curr_datetime = dt.datetime.now().strftime('%Y-%m-%d, %H:%M:%S')
v_start_date = 20250101
print(f'\nParameter input...\n')
print(f'   -> Data as of {curr_datetime}')
print(f'   -> v_start_date: {v_start_date}')


# Connect : TDMDBPR
src_dsn = f'{TDMDBPR_user}/{TDMDBPR_pwd}@{TDMDBPR_host}:{TDMDBPR_port}/{TDMDBPR_db}'
src_conn = oracledb.connect(src_dsn)
src_cur = src_conn.cursor()
query = (f"""
    SELECT /*+PARALLEL(8)*/
        TM_KEY_MTH, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME --, AREA_TYPE, AREA_CD, AREA_NAME
        , MAX(ACTUAL_AS_OF) ACTUAL_AS_OF
        , SUM(CASE WHEN AREA_TYPE = 'C' THEN ACTUAL_SNAP END) C
        , SUM(CASE WHEN AREA_TYPE = 'P' THEN ACTUAL_SNAP END) P
        , SUM(CASE WHEN AREA_TYPE = 'G' THEN ACTUAL_SNAP END) G
        , SUM(CASE WHEN AREA_TYPE = 'H' THEN ACTUAL_SNAP END) H
        , SUM(CASE WHEN AREA_TYPE = 'HH' THEN ACTUAL_SNAP END) HH
        , MAX(PPN_TM) PPN_TM
    FROM GEOSPCAPPO.AGG_PERF_NEWCO
    WHERE METRIC_CD IN (
        'B0R000100' --Total Revenue
        , 'DB0R000100' --Total Revenue : DTAC
        , 'TB0R000100' --Total Revenue : TRUE
        , 'B0R000101' --Mobile Revenue
        , 'DB0R000101' --Mobile Revenue : DTAC
        , 'TB0R000101' --Mobile Revenue : TMH
        , 'B2R000100' --Postpaid Revenue
        , 'B2R010100' --Postpaid Revenue B2C
        , 'DB2R000100' --Postpaid Revenue : DTAC
        , 'DB2R010100' --Postpaid Revenue B2C : DTAC
        , 'TB2R000100' --Postpaid Revenue : TMH
        , 'TB2R010100' --Postpaid Revenue B2C : TMH
        , 'B1R000100' --Prepaid Revenue
        , 'DB1R000100' --Prepaid Revenue : DTAC
        , 'TB1R000100' --Prepaid Revenue : TMH
        , 'TB3R000100' --TOL Revenue
        , 'TB4R000100' --TVS Revenue
        )
    AND AREA_TYPE IN ('C','P','G','H','HH')
    AND TM_KEY_DAY >= {v_start_date}
    GROUP BY TM_KEY_MTH, TM_KEY_DAY, PRODUCT_GRP, COMP_CD, METRIC_CD, METRIC_NAME
""")


try:
    # Get : Timestamp
    src_cur.execute(query)
    rows = src_cur.fetchall()
    chk_src_df = pd.DataFrame.from_records(rows, columns=[x[0] for x in src_cur.description])

    src_cur.close()


except oracledb.DatabaseError as e:
    print(f'\nError with Oracle : {e}')


finally:
    src_conn.close()


Parameter input...

   -> Data as of 2025-05-13, 13:21:02
   -> v_start_date: 20250101


In [3]:
sample_df = chk_src_df.loc[chk_src_df['TM_KEY_DAY']==20250102].copy()

mod_col_list = sample_df.iloc[:, 7:12].columns.tolist()
for col in mod_col_list:
    sample_df[col] = sample_df[col].apply(lambda x: format(x, ',.0f'))

sample_df = sample_df.sort_values(by=['TM_KEY_DAY', 'PRODUCT_GRP', 'METRIC_CD']).reset_index(drop=True)
sample_df

,TM_KEY_MTH,TM_KEY_DAY,PRODUCT_GRP,COMP_CD,METRIC_CD,METRIC_NAME,ACTUAL_AS_OF,C,P,G,H,HH,PPN_TM
0,202501,20250102,All Services,ALL,B0R000100,Total Revenue,20250102.0,"641,115,933","1,253,473,170",nan,nan,nan,2025-05-12 23:06:42
1,202501,20250102,All Services,DTAC,DB0R000100,Total Revenue : DTAC,20250102.0,"301,030,841","301,035,463","301,035,463","300,771,410","300,771,410",2025-05-13 01:44:41
2,202501,20250102,All Services,TRUE,TB0R000100,Total Revenue : TRUE,20250102.0,"340,085,092","952,437,707","335,844,364","335,769,394","335,767,827",2025-05-13 01:44:41
3,202501,20250102,Mobile,ALL,B0R000101,Mobile Revenue,20250102.0,"389,541,647","1,003,155,781",nan,nan,nan,2025-05-12 23:06:42
4,202501,20250102,Mobile,DTAC,DB0R000101,Mobile Revenue : DTAC,20250102.0,"301,030,841","301,035,463",nan,nan,nan,2025-05-12 23:06:42
5,202501,20250102,Mobile,TRUE,TB0R000101,Mobile Revenue : TMH,20250102.0,"88,510,806","702,120,319",nan,nan,nan,2025-05-12 23:06:42
6,202501,20250102,Postpaid,ALL,B2R000100,Postpaid Revenue,20250102.0,"847,437,062","847,437,062","233,715,912","233,452,187","233,452,187",2025-05-13 01:44:41
7,202501,20250102,Postpaid,ALL,B2R010100,Postpaid Revenue B2C,20250102.0,"577,284,228","577,284,228","577,283,416","577,281,093","577,281,093",2025-05-13 01:44:41
8,202501,20250102,Postpaid,DTAC,DB2R000100,Postpaid Revenue : DTAC,20250102.0,"233,715,912","233,715,912","233,715,912","233,452,187","233,452,187",2025-05-13 01:44:41
9,202501,20250102,Postpaid,DTAC,DB2R010100,Postpaid Revenue B2C : DTAC,20250102.0,"97,527,120","97,527,120","97,527,120","97,526,880","97,526,880",2025-05-13 01:44:41


In [4]:
sample_df2 = chk_src_df.loc[chk_src_df['TM_KEY_MTH']==202501].copy()

sample_df2['METRIC_TMP'] = sample_df2['METRIC_NAME'].replace(r' : DTAC| : TRUE| : TMH', '', regex=True)
sample_df2['ALL'] = np.where(sample_df2['COMP_CD']=='ALL', sample_df2['C'], 0)
sample_df2['TRUE'] = np.where(sample_df2['COMP_CD']=='TRUE', sample_df2['C'], 0)
sample_df2['DTAC'] = np.where(sample_df2['COMP_CD']=='DTAC', sample_df2['C'], 0)

sample_df2 = sample_df2.groupby(['TM_KEY_MTH', 'METRIC_TMP']).agg({'ACTUAL_AS_OF':'max', 'ALL':'sum', 'TRUE':'sum', 'DTAC':'sum', 'PPN_TM':'max'})
sample_df2 = sample_df2.fillna(0).sort_values(by=['TM_KEY_MTH', 'METRIC_TMP']).reset_index()

mod_col_list = sample_df2.iloc[:, 3:6].columns.tolist()
for col in mod_col_list:
    sample_df2[col] = sample_df2[col].apply(lambda x: format(x, ',.0f'))

sample_df2

,TM_KEY_MTH,METRIC_TMP,ACTUAL_AS_OF,ALL,TRUE,DTAC,PPN_TM
0,202501,Mobile Revenue,20250131.0,"9,784,556,756","2,444,926,240","7,339,630,516",2025-05-12 23:06:42
1,202501,Postpaid Revenue,20250131.0,"6,470,994,119","3,731,079,534","2,739,914,585",2025-05-13 01:44:41
2,202501,Postpaid Revenue B2C,20250131.0,"5,868,396,046","3,336,770,490","2,531,625,556",2025-05-13 01:44:41
3,202501,Prepaid Revenue,20250131.0,"4,284,739,583","2,444,926,240","1,839,813,343",2025-05-13 01:44:41
4,202501,TOL Revenue,20250128.0,0,"1,597,056,733",0,2025-05-13 01:44:41
5,202501,TVS Revenue,20250131.0,0,"417,316,338",0,2025-05-13 01:44:41
6,202501,Total Revenue,20250131.0,"11,798,929,827","4,459,299,311","7,339,630,516",2025-05-13 01:44:41


In [5]:
''' Total Revenue '''

v_total_metric_list = ['B0R000100', 'TB0R000100', 'DB0R000100']
# v_tm_key_mth = 202501
# v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250401
print(f'\nTotal Revenue...\n')
print(f'   -> v_total_metric_list: {v_total_metric_list}')
# print(f'   -> v_tm_key_day: {v_tm_key_day}')

B0R000100_day_df = chk_src_df.copy()
B0R000100_day_df = B0R000100_day_df.loc[B0R000100_day_df['METRIC_CD'].isin(v_total_metric_list)]
# B0R000100_day_df = B0R000100_day_df.loc[B0R000100_day_df['TM_KEY_MTH']==v_tm_key_mth]
# B0R000100_day_df = B0R000100_day_df.loc[B0R000100_day_df['TM_KEY_DAY']>=v_tm_key_day]

# B0R000100_day_df['METRIC_TMP'] = B0R000100_day_df['METRIC_NAME'].replace(r' : DTAC| : TRUE| : TMH', '', regex=True)
# B0R000100_day_df['ALL'] = np.where(B0R000100_day_df['COMP_CD']=='ALL', B0R000100_day_df['ACTUAL_SNAP'], 0)
# B0R000100_day_df['TRUE'] = np.where(B0R000100_day_df['COMP_CD']=='TRUE', B0R000100_day_df['ACTUAL_SNAP'], 0)
# B0R000100_day_df['DTAC'] = np.where(B0R000100_day_df['COMP_CD']=='DTAC', B0R000100_day_df['ACTUAL_SNAP'], 0)

B0R000100_day_df = B0R000100_day_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'C':'sum', 'P':'sum', 'G':'sum', 'PPN_TM':'max'})
B0R000100_day_df = B0R000100_day_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()
# B0R000100_day_df['ACTUAL_AS_OF'] = B0R000100_day_df['ACTUAL_AS_OF'].astype(str)

B0R000100_day_df['Diff (C - P)'] = B0R000100_day_df['C'] - B0R000100_day_df['P']
B0R000100_day_df = B0R000100_day_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'Diff (C - P)', 'C', 'P', 'G', 'PPN_TM']]

mod_col_list = B0R000100_day_df.iloc[:, 4:8].columns.tolist()
for col in mod_col_list:
    B0R000100_day_df[col] = B0R000100_day_df[col].apply(lambda x: format(x, ',.0f'))
    
B0R000100_day_df


Total Revenue...

   -> v_total_metric_list: ['B0R000100', 'TB0R000100', 'DB0R000100']


,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,Diff (C - P),C,P,G,PPN_TM
0,202501,All Services,B0R000100,Total Revenue,"-838,015,676","11,798,929,827","12,636,945,503",0,2025-05-12 23:06:42
1,202501,All Services,DB0R000100,Total Revenue : DTAC,"2,759,688,949","7,339,630,516","4,579,941,567","4,579,940,849",2025-05-13 01:44:41
2,202501,All Services,TB0R000100,Total Revenue : TRUE,"-3,597,704,625","4,459,299,311","8,057,003,936","4,316,603,800",2025-05-13 01:44:41
3,202502,All Services,B0R000100,Total Revenue,"-1,301,619,275","10,908,511,311","12,210,130,586",0,2025-05-12 23:06:42
4,202502,All Services,DB0R000100,Total Revenue : DTAC,"2,400,545,272","6,776,726,890","4,376,181,618","4,376,180,420",2025-05-13 01:44:41
5,202502,All Services,TB0R000100,Total Revenue : TRUE,"-3,702,164,547","4,131,784,421","7,833,948,969","4,065,402,793",2025-05-13 01:44:41
6,202503,All Services,B0R000100,Total Revenue,"-1,045,192,353","11,535,547,139","12,580,739,492",0,2025-05-12 23:06:42
7,202503,All Services,DB0R000100,Total Revenue : DTAC,"2,734,724,325","7,245,126,495","4,510,402,170","4,510,401,073",2025-05-13 01:44:41
8,202503,All Services,TB0R000100,Total Revenue : TRUE,"-3,779,916,679","4,290,420,644","8,070,337,322","4,300,340,452",2025-05-13 01:44:41
9,202504,All Services,B0R000100,Total Revenue,"-3,743,782,588","8,732,568,997","12,476,351,585",0,2025-05-12 23:06:42


In [8]:
''' Mobile Revenue '''

v_mobile_metric_list = ['B0R000101', 'TB0R000101', 'DB0R000101']
# v_tm_key_mth = 202501
# v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250401
print(f'\nMobile Revenue...\n')
print(f'   -> v_mobile_metric_list: {v_mobile_metric_list}')
# print(f'   -> v_tm_key_day: {v_tm_key_day}')

B0R000101_day_df = chk_src_df.copy()
B0R000101_day_df = B0R000101_day_df.loc[B0R000101_day_df['METRIC_CD'].isin(v_mobile_metric_list)]
# B0R000101_day_df = B0R000101_day_df.loc[B0R000101_day_df['TM_KEY_MTH']==v_tm_key_mth]
# B0R000101_day_df = B0R000101_day_df.loc[B0R000101_day_df['TM_KEY_DAY']>=v_tm_key_day]

B0R000101_day_df = B0R000101_day_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'C':'sum', 'P':'sum', 'G':'sum', 'PPN_TM':'max'})
B0R000101_day_df = B0R000101_day_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()

B0R000101_day_df['Diff (C - P)'] = B0R000101_day_df['C'] - B0R000101_day_df['P']
B0R000101_day_df = B0R000101_day_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'Diff (C - P)', 'C', 'P', 'G', 'PPN_TM']]

mod_col_list = B0R000101_day_df.iloc[:, 4:8].columns.tolist()
for col in mod_col_list:
    B0R000101_day_df[col] = B0R000101_day_df[col].apply(lambda x: format(x, ',.0f'))
    
B0R000101_day_df


Mobile Revenue...

   -> v_mobile_metric_list: ['B0R000101', 'TB0R000101', 'DB0R000101']


,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,Diff (C - P),C,P,G,PPN_TM
0,202501,Mobile,B0R000101,Mobile Revenue,"-966,994,427","9,784,556,756","10,751,551,183",0,2025-05-12 23:06:42
1,202501,Mobile,DB0R000101,Mobile Revenue : DTAC,"2,759,688,949","7,339,630,516","4,579,941,567",0,2025-05-12 23:06:42
2,202501,Mobile,TB0R000101,Mobile Revenue : TMH,"-3,726,683,376","2,444,926,240","6,171,609,616",0,2025-05-12 23:06:42
3,202502,Mobile,B0R000101,Mobile Revenue,"-1,320,085,822","8,991,609,569","10,311,695,392",0,2025-05-12 23:06:42
4,202502,Mobile,DB0R000101,Mobile Revenue : DTAC,"2,400,545,272","6,776,726,890","4,376,181,618",0,2025-05-12 23:06:42
5,202502,Mobile,TB0R000101,Mobile Revenue : TMH,"-3,720,631,095","2,214,882,680","5,935,513,774",0,2025-05-12 23:06:42
6,202503,Mobile,B0R000101,Mobile Revenue,"-966,757,089","9,704,486,450","10,671,243,538",0,2025-05-12 23:06:42
7,202503,Mobile,DB0R000101,Mobile Revenue : DTAC,"2,734,724,325","7,245,126,495","4,510,402,170",0,2025-05-12 23:06:42
8,202503,Mobile,TB0R000101,Mobile Revenue : TMH,"-3,701,481,414","2,459,359,955","6,160,841,369",0,2025-05-12 23:06:42
9,202504,Mobile,B0R000101,Mobile Revenue,"-3,761,232,310","6,798,572,586","10,559,804,895",0,2025-05-12 23:06:42


In [9]:
''' Prepaid Revenue '''

v_prepaid_metric_list = ['B1R000100', 'TB1R000100', 'DB1R000100']
# v_tm_key_mth = 202501
# v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250401
print(f'\nPrepaid Revenue...\n')
print(f'   -> v_prepaid_metric_list: {v_prepaid_metric_list}')
# print(f'   -> v_tm_key_day: {v_tm_key_day}')

B1R000100_day_df = chk_src_df.copy()
B1R000100_day_df = B1R000100_day_df.loc[B1R000100_day_df['METRIC_CD'].isin(v_prepaid_metric_list)]
# B1R000100_day_df = B1R000100_day_df.loc[B1R000100_day_df['TM_KEY_MTH']==v_tm_key_mth]
# B1R000100_day_df = B1R000100_day_df.loc[B1R000100_day_df['TM_KEY_DAY']>=v_tm_key_day]

B1R000100_day_df = B1R000100_day_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'C':'sum', 'P':'sum', 'G':'sum', 'PPN_TM':'max'})
B1R000100_day_df = B1R000100_day_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()

B1R000100_day_df['Diff (C - P)'] = B1R000100_day_df['C'] - B1R000100_day_df['P']
B1R000100_day_df = B1R000100_day_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'Diff (C - P)', 'C', 'P', 'G', 'PPN_TM']]

mod_col_list = B1R000100_day_df.iloc[:, 4:8].columns.tolist()
for col in mod_col_list:
    B1R000100_day_df[col] = B1R000100_day_df[col].apply(lambda x: format(x, ',.0f'))
    
B1R000100_day_df


Prepaid Revenue...

   -> v_prepaid_metric_list: ['B1R000100', 'TB1R000100', 'DB1R000100']


,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,Diff (C - P),C,P,G,PPN_TM
0,202501,Prepaid,B1R000100,Prepaid Revenue,"4,182,519","4,284,739,583","4,280,557,064","4,271,236,462",2025-05-13 01:44:41
1,202501,Prepaid,DB1R000100,Prepaid Revenue : DTAC,"-213,638","1,839,813,343","1,840,026,982","1,840,026,982",2025-05-13 01:44:41
2,202501,Prepaid,TB1R000100,Prepaid Revenue : TMH,"4,396,158","2,444,926,240","2,440,530,082","2,431,209,481",2025-05-13 01:44:41
3,202502,Prepaid,B1R000100,Prepaid Revenue,"5,627,755","3,863,528,742","3,857,900,987","3,815,785,751",2025-05-13 01:44:41
4,202502,Prepaid,DB1R000100,Prepaid Revenue : DTAC,"-172,089","1,648,646,062","1,648,818,151","1,648,818,151",2025-05-13 01:44:41
5,202502,Prepaid,TB1R000100,Prepaid Revenue : TMH,"5,799,844","2,214,882,680","2,209,082,836","2,166,967,599",2025-05-13 01:44:41
6,202503,Prepaid,B1R000100,Prepaid Revenue,"6,991,027","4,234,712,526","4,227,721,499","4,166,278,548",2025-05-13 01:44:41
7,202503,Prepaid,DB1R000100,Prepaid Revenue : DTAC,"-81,479","1,775,352,571","1,775,434,050","1,775,434,050",2025-05-13 01:44:41
8,202503,Prepaid,TB1R000100,Prepaid Revenue : TMH,"7,072,506","2,459,359,955","2,452,287,449","2,390,844,498",2025-05-13 01:44:41
9,202504,Prepaid,B1R000100,Prepaid Revenue,"6,401,445","4,053,942,394","4,047,540,949","3,981,837,547",2025-05-13 01:44:41


In [12]:
# ''' Postpaid Revenue '''

# v_postpaid_metric_list = ['B2R000100', 'TB2R000100', 'DB2R000100']
# # v_tm_key_mth = 202501
# # v_tm_key_day = int(str(curr_mth)+str('01'))
# # v_tm_key_day = 20250401
# print(f'\nPostpaid Revenue...\n')
# print(f'   -> v_postpaid_metric_list: {v_postpaid_metric_list}')
# # print(f'   -> v_tm_key_day: {v_tm_key_day}')

# B2R000100_day_df = chk_src_df.copy()
# B2R000100_day_df = B2R000100_day_df.loc[B2R000100_day_df['METRIC_CD'].isin(v_postpaid_metric_list)]
# # B2R000100_day_df = B2R000100_day_df.loc[B2R000100_day_df['TM_KEY_MTH']==v_tm_key_mth]
# # B2R000100_day_df = B2R000100_day_df.loc[B2R000100_day_df['TM_KEY_DAY']>=v_tm_key_day]

# B2R000100_day_df = B2R000100_day_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'C':'sum', 'P':'sum', 'G':'sum', 'PPN_TM':'max'})
# B2R000100_day_df = B2R000100_day_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()

# B2R000100_day_df['Diff (C - P)'] = B2R000100_day_df['C'] - B2R000100_day_df['P']
# B2R000100_day_df = B2R000100_day_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'Diff (C - P)', 'C', 'P', 'G', 'PPN_TM']]

# mod_col_list = B2R000100_day_df.iloc[:, 4:8].columns.tolist()
# for col in mod_col_list:
#     B2R000100_day_df[col] = B2R000100_day_df[col].apply(lambda x: format(x, ',.0f'))
    
# B2R000100_day_df

In [14]:
# ''' Postpaid Revenue B2C '''

# v_postpaid_b2c_metric_list = ['B2R010100', 'TB2R010100', 'DB2R010100']
# # v_tm_key_mth = 202501
# # v_tm_key_day = int(str(curr_mth)+str('01'))
# # v_tm_key_day = 20250401
# print(f'\nPostpaid Revenue B2C...\n')
# print(f'   -> v_postpaid_b2c_metric_list: {v_postpaid_b2c_metric_list}')
# # print(f'   -> v_tm_key_day: {v_tm_key_day}')

# B2R010100_day_df = chk_src_df.copy()
# B2R010100_day_df = B2R010100_day_df.loc[B2R010100_day_df['METRIC_CD'].isin(v_postpaid_b2c_metric_list)]
# # B2R010100_day_df = B2R010100_day_df.loc[B2R010100_day_df['TM_KEY_MTH']==v_tm_key_mth]
# # B2R010100_day_df = B2R010100_day_df.loc[B2R010100_day_df['TM_KEY_DAY']>=v_tm_key_day]

# B2R010100_day_df = B2R010100_day_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'C':'sum', 'P':'sum', 'G':'sum', 'PPN_TM':'max'})
# B2R010100_day_df = B2R010100_day_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()

# B2R010100_day_df['Diff (C - P)'] = B2R010100_day_df['C'] - B2R010100_day_df['P']
# B2R010100_day_df = B2R010100_day_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'Diff (C - P)', 'C', 'P', 'G', 'PPN_TM']]

# mod_col_list = B2R010100_day_df.iloc[:, 4:8].columns.tolist()
# for col in mod_col_list:
#     B2R010100_day_df[col] = B2R010100_day_df[col].apply(lambda x: format(x, ',.0f'))
    
# B2R010100_day_df

In [15]:
''' TOL Revenue '''

v_tol_metric_list = ['TB3R000100']
# v_tm_key_mth = 202501
# v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250401
print(f'\nTOL Revenue...\n')
print(f'   -> v_tol_metric_list: {v_tol_metric_list}')
# print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB3R000100_day_df = chk_src_df.copy()
TB3R000100_day_df = TB3R000100_day_df.loc[TB3R000100_day_df['METRIC_CD'].isin(v_tol_metric_list)]
# TB3R000100_day_df = TB3R000100_day_df.loc[TB3R000100_day_df['TM_KEY_MTH']==v_tm_key_mth]
# TB3R000100_day_df = TB3R000100_day_df.loc[TB3R000100_day_df['TM_KEY_DAY']>=v_tm_key_day]

TB3R000100_day_df = TB3R000100_day_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'C':'sum', 'P':'sum', 'G':'sum', 'PPN_TM':'max'})
TB3R000100_day_df = TB3R000100_day_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()

TB3R000100_day_df['Diff (C - P)'] = TB3R000100_day_df['C'] - TB3R000100_day_df['P']
TB3R000100_day_df = TB3R000100_day_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'Diff (C - P)', 'C', 'P', 'G', 'PPN_TM']]

mod_col_list = TB3R000100_day_df.iloc[:, 4:8].columns.tolist()
for col in mod_col_list:
    TB3R000100_day_df[col] = TB3R000100_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB3R000100_day_df


TOL Revenue...

   -> v_tol_metric_list: ['TB3R000100']


,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,Diff (C - P),C,P,G,PPN_TM
0,202501,TOL,TB3R000100,TOL Revenue,"4,545,013","1,597,056,733","1,592,511,720","1,592,511,720",2025-05-13 01:44:41
1,202502,TOL,TB3R000100,TOL Revenue,"4,254,776","1,609,673,918","1,605,419,142","1,605,419,142",2025-05-13 01:44:41
2,202503,TOL,TB3R000100,TOL Revenue,"4,185,979","1,616,442,724","1,612,256,745","1,612,256,745",2025-05-13 01:44:41
3,202504,TOL,TB3R000100,TOL Revenue,"4,079,823","1,616,821,911","1,612,742,089","1,612,742,089",2025-05-13 01:44:41
4,202505,TOL,TB3R000100,TOL Revenue,"4,687,737","598,866,240","594,178,503","594,178,503",2025-05-13 01:44:41


In [16]:
''' TVS Revenue '''

v_tvs_metric_list = ['TB4R000100']
# v_tm_key_mth = 202501
# v_tm_key_day = int(str(curr_mth)+str('01'))
# v_tm_key_day = 20250401
print(f'\nTVS Revenue...\n')
print(f'   -> v_tvs_metric_list: {v_tvs_metric_list}')
# print(f'   -> v_tm_key_day: {v_tm_key_day}')

TB4R000100_day_df = chk_src_df.copy()
TB4R000100_day_df = TB4R000100_day_df.loc[TB4R000100_day_df['METRIC_CD'].isin(v_tvs_metric_list)]
# TB4R000100_day_df = TB4R000100_day_df.loc[TB4R000100_day_df['TM_KEY_MTH']==v_tm_key_mth]
# TB4R000100_day_df = TB4R000100_day_df.loc[TB4R000100_day_df['TM_KEY_DAY']>=v_tm_key_day]

TB4R000100_day_df = TB4R000100_day_df.groupby(['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME']).agg({'C':'sum', 'P':'sum', 'G':'sum', 'PPN_TM':'max'})
TB4R000100_day_df = TB4R000100_day_df.fillna(0).sort_values(by=['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD']).reset_index()

TB4R000100_day_df['Diff (C - P)'] = TB4R000100_day_df['C'] - TB4R000100_day_df['P']
TB4R000100_day_df = TB4R000100_day_df[['TM_KEY_MTH', 'PRODUCT_GRP', 'METRIC_CD', 'METRIC_NAME', 'Diff (C - P)', 'C', 'P', 'G', 'PPN_TM']]

mod_col_list = TB4R000100_day_df.iloc[:, 4:8].columns.tolist()
for col in mod_col_list:
    TB4R000100_day_df[col] = TB4R000100_day_df[col].apply(lambda x: format(x, ',.0f'))
    
TB4R000100_day_df


TVS Revenue...

   -> v_tvs_metric_list: ['TB4R000100']


,TM_KEY_MTH,PRODUCT_GRP,METRIC_CD,METRIC_NAME,Diff (C - P),C,P,G,PPN_TM
0,202501,TVS,TB4R000100,TVS Revenue,"124,433,738","417,316,338","292,882,600","292,882,600",2025-05-13 01:44:41
1,202502,TVS,TB4R000100,TVS Revenue,"14,211,771","307,227,824","293,016,052","293,016,052",2025-05-13 01:44:41
2,202503,TVS,TB4R000100,TVS Revenue,"-82,621,243","214,617,965","297,239,208","297,239,208",2025-05-13 01:44:41
3,202504,TVS,TB4R000100,TVS Revenue,"13,369,899","317,174,500","303,804,601","303,804,601",2025-05-13 01:44:41
4,202505,TVS,TB4R000100,TVS Revenue,"5,145,565","76,848,520","71,702,955","71,702,955",2025-05-13 01:44:41
